## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
X = application_df.drop(columns = ['EIN', 'NAME'])

In [3]:
# Determine the number of unique values in each column.
application_df.nunique()

EIN                       34299
NAME                      19568
APPLICATION_TYPE             17
AFFILIATION                   6
CLASSIFICATION               71
USE_CASE                      5
ORGANIZATION                  4
STATUS                        2
INCOME_AMT                    9
SPECIAL_CONSIDERATIONS        2
ASK_AMT                    8747
IS_SUCCESSFUL                 2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
X['APPLICATION_TYPE'].value_counts()


APPLICATION_TYPE
T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: count, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

# Replace in dataframe
for app in application_types_to_replace:
    X['APPLICATION_TYPE'] = X['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
X['APPLICATION_TYPE'].value_counts()

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
classification_df = pd.DataFrame(X['CLASSIFICATION'].value_counts())
classification_df.reset_index()

,CLASSIFICATION,count
0,C1000,17326
1,C2000,6074
2,C1200,4837
3,C3000,1918
4,C2100,1883
...,...,...
66,C4120,1
67,C8210,1
68,C2561,1
69,C4500,1


In [7]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = []

for index, row in classification_df.iterrows():
    if row["count"] < 500:
        classifications_to_replace.append(index)

# Replace in dataframe
for cls in classifications_to_replace:
    X['CLASSIFICATION'] = X['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
X['CLASSIFICATION'].value_counts()

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: count, dtype: int64

In [8]:
X.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
affiliation_dummies = pd.get_dummies(X["AFFILIATION"])

use_case_dummies = pd.get_dummies(X["USE_CASE"])

organization_dummies = pd.get_dummies(X["ORGANIZATION"])

application_dummies = pd.get_dummies(X["APPLICATION_TYPE"])

classification_dummies = pd.get_dummies(X["CLASSIFICATION"])

income_amt_dummies = pd.get_dummies(X["INCOME_AMT"])


In [10]:
X = pd.concat([X, affiliation_dummies, use_case_dummies, organization_dummies, application_dummies, classification_dummies, income_amt_dummies], axis =1)

X.drop(columns=["AFFILIATION", "USE_CASE", "ORGANIZATION", "APPLICATION_TYPE", "CLASSIFICATION", "INCOME_AMT"], inplace=True)


In [11]:
X.columns

Index(['STATUS', 'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'IS_SUCCESSFUL',
       'CompanySponsored', 'Family/Parent', 'Independent', 'National', 'Other',
       'Regional', 'CommunityServ', 'Heathcare', 'Other', 'Preservation',
       'ProductDev', 'Association', 'Co-operative', 'Corporation', 'Trust',
       'Other', 'T10', 'T19', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'C1000',
       'C1200', 'C2000', 'C2100', 'C3000', 'C7000', 'Other', '0', '1-9999',
       '10000-24999', '100000-499999', '10M-50M', '1M-5M', '25000-99999',
       '50M+', '5M-10M'],
      dtype='object')

In [12]:
X["SPECIAL_CONSIDERATIONS"].value_counts()

SPECIAL_CONSIDERATIONS
N    34272
Y       27
Name: count, dtype: int64

In [13]:
def encode_special_considerations(encode_special_considerations):
  if encode_special_considerations == "Y":
    return 1
  else:
    return 0

X["SPECIAL_CONSIDERATIONS"] = X["SPECIAL_CONSIDERATIONS"].apply(encode_special_considerations)
X.head()

,STATUS,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,CompanySponsored,Family/Parent,Independent,National,Other,Regional,...,Other,0,1-9999,10000-24999,100000-499999,10M-50M,1M-5M,25000-99999,50M+,5M-10M
0,1,0,5000,1,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,1,0,108590,1,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,1,0,5000,0,True,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
3,1,0,6692,1,True,False,False,False,False,False,...,False,False,False,True,False,False,False,False,False,False
4,1,0,142590,1,False,False,True,False,False,False,...,False,False,False,False,True,False,False,False,False,False


In [14]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 44 columns):
 #   Column                  Non-Null Count  Dtype
---  ------                  --------------  -----
 0   STATUS                  34299 non-null  int64
 1   SPECIAL_CONSIDERATIONS  34299 non-null  int64
 2   ASK_AMT                 34299 non-null  int64
 3   IS_SUCCESSFUL           34299 non-null  int64
 4   CompanySponsored        34299 non-null  bool 
 5   Family/Parent           34299 non-null  bool 
 6   Independent             34299 non-null  bool 
 7   National                34299 non-null  bool 
 8   Other                   34299 non-null  bool 
 9   Regional                34299 non-null  bool 
 10  CommunityServ           34299 non-null  bool 
 11  Heathcare               34299 non-null  bool 
 12  Other                   34299 non-null  bool 
 13  Preservation            34299 non-null  bool 
 14  ProductDev              34299 non-null  bool 
 15  Association        

In [15]:
# Split our preprocessed data into our features and target arrays
y = X["IS_SUCCESSFUL"]
X = X.drop(columns = "IS_SUCCESSFUL")

# Split the preprocessed data into a training and testing dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify = y)

In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

 # Input Layer
nn_model.add(tf.keras.layers.Dense(units=9, activation="relu", input_dim=43))

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=9, activation="relu"))

# Second hidden layer
#nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 9)                 396       
                                                                 
 dense_1 (Dense)             (None, 9)                 90        
                                                                 
 dense_2 (Dense)             (None, 1)                 10        
                                                                 
Total params: 496 (1.94 KB)
Trainable params: 496 (1.94 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [25]:
#Create the callback to save model weights every 5 epochs
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint_callback = ModelCheckpoint(filepath='weights.{epoch:02d}.h5',
                                      save_weights_only=True,
                                      save_freq=5)

In [26]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
804/804 [==============================] - 5s 5ms/step - loss: 0.5433 - accuracy: 0.7357
Epoch 2/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5430 - accuracy: 0.7364
Epoch 3/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5426 - accuracy: 0.7366
Epoch 4/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5431 - accuracy: 0.7369
Epoch 5/50
804/804 [==============================] - 4s 5ms/step - loss: 0.5427 - accuracy: 0.7360
Epoch 6/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5428 - accuracy: 0.7366
Epoch 7/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5425 - accuracy: 0.7358
Epoch 8/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5424 - accuracy: 0.7362
Epoch 9/50
804/804 [==============================] - 4s 4ms/step - loss: 0.5425 - accuracy: 0.7363
Epoch 10/50
804/804 [==============================] - 3s 4ms/step - loss: 0.5426 - accuracy: 0.7364

In [27]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5562 - accuracy: 0.7214 - 454ms/epoch - 2ms/step
Loss: 0.5562271475791931, Accuracy: 0.7213994264602661


In [28]:
# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [29]:
nn_model.save("AlphabetSoupCharity.keras")